In [11]:
# import libraries
# import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import f1_score, matthews_corrcoef, cohen_kappa_score, adjusted_rand_score, normalized_mutual_info_score, confusion_matrix
from scipy.spatial.distance import jensenshannon
import numpy as np

In [12]:

# calculate the cell type distribution in terms of percentages for predicted and true phenotypes
def calculate_cell_type_distribution(df, predictions, true_label):
    """
    Calculate the cell type distribution in terms of percentages for predicted and true phenotypes.

    Parameters:
        df (pd.DataFrame): DataFrame containing the predictions.

    Returns:
        pd.DataFrame: DataFrame containing the cell type distribution for predicted and true phenotypes.
    """
    # Get the counts and percentages for predicted phenotypes
    counts_predicted = predictions.value_counts()
    percentages_predicted = counts_predicted / counts_predicted.sum() * 100
    predicted_distribution = pd.DataFrame({'cell_type': counts_predicted.index, 'predicted_percentage': percentages_predicted.values})

    # Get the counts and percentages for true phenotypes
    counts_true = true_label.value_counts()
    percentages_true = counts_true / counts_true.sum() * 100
    true_distribution = pd.DataFrame({'cell_type': counts_true.index, 'true_percentage': percentages_true.values})

    # Merge the two distributions
    distribution_df = pd.merge(predicted_distribution, true_distribution, on='cell_type', how='outer').fillna(0)

    return distribution_df

# calcualte r2 and pearson correlation for the predicted and true phenotypes
def calculate_r2_and_pearson(df):
    """
    Calculate R2 and Pearson correlation for the predicted and true phenotypes.

    Parameters:
        df (pd.DataFrame): DataFrame containing the predictions.

    Returns:
        tuple: R2 and Pearson correlation values.
    """
    # Calculate R2
    r2 = df['predicted_percentage'].corr(df['true_percentage']) ** 2

    # Calculate Pearson correlation
    pearson_corr = df['predicted_percentage'].corr(df['true_percentage'])

    return r2, pearson_corr

# parse the tree file to get label -> ancestor path mapping
def parse_tree_file(file_path: str) -> dict:
    """Parses a 2-space-indented tree file and returns label -> ancestor path mapping."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    label_to_ancestors = {}
    path_stack = []

    for line in lines:
        stripped = line.lstrip()
        indent = len(line) - len(stripped)
        level = indent // 2  # Assumes 2 spaces per indent

        path_stack = path_stack[:level]
        path_stack.append(stripped.strip())

        if level >= 2:
            label = path_stack[-1]
            label_to_ancestors[label] = path_stack[:-1]  # from root to parent

    return label_to_ancestors

# calculate hierarchical F1 score given ground truths, predictions, and an ancestor map
def hierarchical_f1_score(y_true, y_pred, ancestor_map):
    """Computes hierarchical F1 given ground truths, predictions, and an ancestor map."""
    scores = []

    for true_label, pred_label in zip(y_true, y_pred):
        true_anc = set(ancestor_map.get(true_label, []))
        pred_anc = set(ancestor_map.get(pred_label, []))

        if not true_anc or not pred_anc:
            scores.append(0.0)
            continue

        intersection = true_anc & pred_anc
        precision = len(intersection) / len(pred_anc)
        recall = len(intersection) / len(true_anc)
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0.0
        scores.append(f1)

    return sum(scores) / len(scores)

#get level_2 and level_1 for a predicted phenotype
def map_predicted_to_levels(predicted_phenotype):
    return hierarchy_mappings.get(predicted_phenotype, {'level_2_cell_type': None, 'level_1_cell_type': None})

# G-Mean (Geometric Mean of per-class sensitivity)
def gmean_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    with np.errstate(divide='ignore', invalid='ignore'):
        sensitivity = np.diag(cm) / cm.sum(axis=1)
        gmean = np.prod(sensitivity[sensitivity > 0]) ** (1 / len(sensitivity[sensitivity > 0]))
    return gmean

# get the average train and inference time
def calculate_time(time_path):

    with open(time_path, "r") as f:
        fold_times = f.readlines()
    fold_times = [line.strip() for line in fold_times if line.strip()]

    # get average train_time and inference_time and add to the results dataframe
    fold_times_dict = {}
    for line in fold_times:
        line = line.replace(',', ':') 
        parts = line.split(":")
        if len(parts) == 2:
            fold_name = parts[0].strip()
            fold_time = parts[1].strip()
            fold_time = float(fold_time.split()[0])  # Assuming the time is in seconds
            fold_times_dict[fold_name] = {'time': fold_time}

    # in fold times_dict, add the average time for each fold
    fold_times_df = pd.DataFrame.from_dict(fold_times_dict, orient='index')
    fold_times_df['fold'] = fold_times_df.index
    fold_times_df.reset_index(drop=True, inplace=True)

    # split fold name on the second " " into two new cols
    fold_times_df[['fold_name', 'fold_num', 'event']] = fold_times_df['fold'].astype(str).str.split(pat=' ', n=2, expand=True) 
    #fold_times_df["time"] = fold_times_df["time"].astype(float)

    # return the average for all train_time and inference_time
    avg_train_time = fold_times_df[
        (fold_times_df['event'] == 'train_time') | (fold_times_df['event'] == "training_time")
        ]['time'].mean()
    avg_inference_time = fold_times_df[
        (fold_times_df['event'] != 'train_time') | (fold_times_df['event'] != "training_time")
    ]['time'].mean()

    return avg_train_time, avg_inference_time

In [3]:
def calculate_metrics_for_methods(methods, levels, base_path):
    all_results = []

    for method in tqdm(methods, desc="Methods"):
        try:
            for level in levels:
                print(f"Processing method: {method}, level: {level}")
                path = os.path.join(base_path, method, "level3")
                
                if os.path.exists(path):
                    files = [f for f in os.listdir(path) if f.startswith("predictions") and f.endswith(".csv")]
                else:
                    print(f"Path {path} does not exist. Skipping...")
                    files = []

                for file in files:
                    file_path = os.path.join(path, file)
                    if not os.path.exists(file_path):
                        print(f"File {file_path} does not exist. Skipping...")
                        continue
                    df = pd.read_csv(file_path)
                    fold_name = os.path.splitext(file)[0]
                    if 'cell_type' in df.columns:
                        df.rename(columns={'cell_type': 'true_phenotype'}, inplace=True)
                        df.to_csv(file_path, index=False)
                        print(f"Updated {file} to rename 'cell_type' to 'true_phenotype'.")
                    else:
                        continue                        
                    if level == 'level_3':
                        true_label = df['true_phenotype']
                        predictions = df["predicted_phenotype"]
                    else:
                        df[f'mapped_{level}_cell_type'] = df['predicted_phenotype'].apply(lambda x: map_predicted_to_levels(x)[f'{level}_cell_type'])
                        mask_na = df[f'mapped_{level}_cell_type'].isna()
                        df.loc[mask_na, f'mapped_{level}_cell_type'] = df.loc[mask_na, 'predicted_phenotype']

                        if f'{level}_cell_type' not in df.columns:
                            true_label = df['true_phenotype'].apply(lambda x: map_predicted_to_levels(x)[f'{level}_cell_type'])
                        else:
                            true_label = df[f'{level}_cell_type']
                        
                        predictions = df[f'mapped_{level}_cell_type'] 

                    f1 = f1_score(true_label, predictions, average='weighted')
                    accuracy = (true_label == predictions).mean()
                    macro_f1 = f1_score(true_label, predictions, average='macro')
                    mcc = matthews_corrcoef(true_label, predictions)
                    kappa = cohen_kappa_score(true_label, predictions)

                    if level == 'level_3':
                        ancestor_map = parse_tree_file("cell_type_hierarchy.txt")
                        hierarchical_f1 = hierarchical_f1_score(true_label, predictions, ancestor_map)
                    else:
                        hierarchical_f1 = None
                    
                    cell_type_distribution = calculate_cell_type_distribution(df, predictions, true_label)
                    r2, pearson_corr = calculate_r2_and_pearson(cell_type_distribution)
                    ari = adjusted_rand_score(true_label, predictions)
                    nmi = normalized_mutual_info_score(true_label, predictions)
                    kl_divergence = sum(cell_type_distribution['predicted_percentage'] / 100 * np.log2((cell_type_distribution['predicted_percentage'] + 1e-9) / (cell_type_distribution['true_percentage'] + 1e-9)))
                    scaled_kl_mean = 1 / (1 + kl_divergence)
                    jensen = jensenshannon(cell_type_distribution['predicted_percentage'] / 100, cell_type_distribution['true_percentage'] / 100, base=2)
                    jensen_scaled = 1 - jensen
                    g_mean = gmean_score(true_label, predictions)
                    
                    times_path1 = os.path.join(path, "fold_times.txt")
                    times_path2 = os.path.join(base_path,method, "fold_times.txt")
                    if os.path.exists(times_path1):
                        average_train_time, average_inference_time = calculate_time(times_path1)
                    elif os.path.exists(times_path2):
                        average_train_time, average_inference_time = calculate_time(times_path2)
                    else:
                        average_train_time = None
                        average_inference_time = None

                    all_results.append({
                        'method': method,
                        'level': level,
                        'fold': fold_name,
                        'f1_weighted': f1,
                        'hierarchical_f1': hierarchical_f1,
                        'accuracy': accuracy,
                        'macro_f1': macro_f1,
                        'g_mean': g_mean,
                        'mcc': mcc,
                        'kappa': kappa,
                        'r2': r2,
                        'pearson_corr': pearson_corr,
                        'ari': ari,
                        'nmi': nmi,
                        'jsd': jensen,
                        'jsd_scaled': jensen_scaled,
                        'kl_divergence': kl_divergence,
                        'kl_scaled': scaled_kl_mean,
                        'train_time': average_train_time,
                        'inference_time':average_inference_time
                    })
            print(f"Finished processing method: {method}")
        except Exception as e:
            print(f"Error encountered for method {method}: {e}. Skipping this method.")
            continue

    return pd.DataFrame(all_results)


In [6]:
def calculate_metrics_for_methods(methods, levels, base_path):
    all_results = []

    for method in tqdm(methods, desc="Methods"):
        try:
            for level in levels:
                print(f"Processing method: {method}, level: {level}")
                path = os.path.join(base_path, method, "level3")
                
                if os.path.exists(path):
                    files = [f for f in os.listdir(path) if f.startswith("predictions") and f.endswith(".csv")]
                else:
                    print(f"Path {path} does not exist. Skipping...")
                    files = []

                for file in files:
                    file_path = os.path.join(path, file)
                    if not os.path.exists(file_path):
                        print(f"File {file_path} does not exist. Skipping...")
                        continue
                    df = pd.read_csv(file_path)
                    fold_name = os.path.splitext(file)[0]
                    
                    if level == 'level_3':
                        true_label = df['true_phenotype']
                        predictions = df["predicted_phenotype"]
                    else:
                        df[f'mapped_{level}_cell_type'] = df['predicted_phenotype'].apply(lambda x: map_predicted_to_levels(x)[f'{level}_cell_type'])
                        mask_na = df[f'mapped_{level}_cell_type'].isna()
                        df.loc[mask_na, f'mapped_{level}_cell_type'] = df.loc[mask_na, 'predicted_phenotype']

                        if f'{level}_cell_type' not in df.columns:
                            true_label = df['true_phenotype'].apply(lambda x: map_predicted_to_levels(x)[f'{level}_cell_type'])
                        else:
                            true_label = df[f'{level}_cell_type']
                        
                        predictions = df[f'mapped_{level}_cell_type'] 

                    f1 = f1_score(true_label, predictions, average='weighted')
                    accuracy = (true_label == predictions).mean()
                    macro_f1 = f1_score(true_label, predictions, average='macro')
                    mcc = matthews_corrcoef(true_label, predictions)
                    kappa = cohen_kappa_score(true_label, predictions)

                    if level == 'level_3':
                        ancestor_map = parse_tree_file("cell_type_hierarchy.txt")
                        hierarchical_f1 = hierarchical_f1_score(true_label, predictions, ancestor_map)
                    else:
                        hierarchical_f1 = None
                    
                    cell_type_distribution = calculate_cell_type_distribution(df, predictions, true_label)
                    r2, pearson_corr = calculate_r2_and_pearson(cell_type_distribution)
                    ari = adjusted_rand_score(true_label, predictions)
                    nmi = normalized_mutual_info_score(true_label, predictions)
                    kl_divergence = sum(cell_type_distribution['predicted_percentage'] / 100 * np.log2((cell_type_distribution['predicted_percentage'] + 1e-9) / (cell_type_distribution['true_percentage'] + 1e-9)))
                    scaled_kl_mean = 1 / (1 + kl_divergence)
                    jensen = jensenshannon(cell_type_distribution['predicted_percentage'] / 100, cell_type_distribution['true_percentage'] / 100, base=2)
                    jensen_scaled = 1 - jensen
                    g_mean = gmean_score(true_label, predictions)
                    
                    times_path1 = os.path.join(path, "fold_times.txt")
                    times_path2 = os.path.join(base_path,method, "fold_times.txt")
                    if os.path.exists(times_path1):
                        average_train_time, average_inference_time = calculate_time(times_path1)
                    elif os.path.exists(times_path2):
                        average_train_time, average_inference_time = calculate_time(times_path2)
                    else:
                        average_train_time = None
                        average_inference_time = None

                    all_results.append({
                        'method': method,
                        'level': level,
                        'fold': fold_name,
                        'f1_weighted': f1,
                        'hierarchical_f1': hierarchical_f1,
                        'accuracy': accuracy,
                        'macro_f1': macro_f1,
                        'g_mean': g_mean,
                        'mcc': mcc,
                        'kappa': kappa,
                        'r2': r2,
                        'pearson_corr': pearson_corr,
                        'ari': ari,
                        'nmi': nmi,
                        'jsd': jensen,
                        'jsd_scaled': jensen_scaled,
                        'kl_divergence': kl_divergence,
                        'kl_scaled': scaled_kl_mean,
                        'train_time': average_train_time,
                        'inference_time':average_inference_time
                    })
            print(f"Finished processing method: {method}")
        except Exception as e:
            print(f"Error encountered for method {method}: {e}. Skipping this method.")
            continue

    return pd.DataFrame(all_results)


In [13]:
#read the hierarchy mappings from the file
with open('hierarchy_mappings.pkl', 'rb') as f:
    import pickle
    hierarchy_mappings = pickle.load(f)

In [5]:
base_path = "../results/IMMUcan"
levels = ['level_1', 'level_2', 'level_3']

#make a list of folders in the base_path
methods = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]

# Calculate metrics for all methods and levels
results = calculate_metrics_for_methods(methods, levels, base_path)

# Get the average of the metrics for each method and level, excluding the 'fold' column
average_results = results.drop(columns=["fold"]).groupby(['method', 'level'], as_index=False).mean()
# Get the standard deviation of the metrics for each method and level
std_results = results.drop(columns=["fold"]).groupby(['method', 'level']).std().reset_index()
# Merge the average and standard deviation results
final_results = pd.merge(average_results, std_results, on=['method', 'level'], suffixes=('_mean', '_std'))

# calculate a stability metric for the methods where s = (1 - std/stability_thresh)
stability_thresh = 0.2
final_results['stability'] = 1 - (final_results['f1_weighted_std'] / stability_thresh)
# set to 0 if negative
final_results.loc[final_results['stability'] < 0, 'stability'] = 0

# Save the final results to a CSV file with ';' as the separator
final_results.to_csv(os.path.join(base_path, "final_results.csv"), index=False, sep=';')

Methods:   0%|          | 0/34 [00:00<?, ?it/s]

Processing method: phenograph_80, level: level_1
Processing method: phenograph_80, level: level_2
Processing method: phenograph_80, level: level_3


Methods:   3%|▎         | 1/34 [03:09<1:44:27, 189.94s/it]

Finished processing method: phenograph_80
Processing method: maps, level: level_1
Processing method: maps, level: level_2
Processing method: maps, level: level_3


Methods:   6%|▌         | 2/34 [04:49<1:13:04, 137.03s/it]

...

Finished processing method: nimbus_flowsom
Processing method: ribca, level: level_1
Processing method: ribca, level: level_2
Processing method: ribca, level: level_3


Methods:  97%|█████████▋| 33/34 [1:15:33<02:03, 123.26s/it]

Finished processing method: ribca
Processing method: CellLENS_Lite, level: level_1
Processing method: CellLENS_Lite, level: level_2
Processing method: CellLENS_Lite, level: level_3


Methods: 100%|██████████| 34/34 [1:18:13<00:00, 138.05s/it]

Finished processing method: CellLENS_Lite


In [6]:
# calculate in a new col overall score from all the scores and sort descending
final_results['overall_score_mean'] = (final_results['f1_weighted_mean'] + final_results['accuracy_mean'] + 
                                   final_results['macro_f1_mean'] + final_results['mcc_mean'] + 
                                   final_results['kappa_mean'] + final_results['r2_mean'] + 
                                   final_results['pearson_corr_mean'] + final_results['ari_mean'] + 
                                   final_results['stability']) / 9

In [7]:
final_results

,method,level,f1_weighted_mean,hierarchical_f1_mean,accuracy_mean,macro_f1_mean,g_mean_mean,mcc_mean,kappa_mean,r2_mean,...,ari_std,nmi_std,jsd_std,jsd_scaled_std,kl_divergence_std,kl_scaled_std,train_time_std,inference_time_std,stability,overall_score_mean
0,CellLENS_Lite,level_1,0.795586,NaN,0.811839,0.593905,0.724235,0.696093,0.691456,0.961711,...,0.023751,0.021674,0.024221,0.024221,0.018878,0.016816,NaN,0.0,0.946108,0.782111
1,CellLENS_Lite,level_2,0.764736,NaN,0.785496,0.592056,0.663559,0.683480,0.679137,0.983864,...,0.022771,0.016064,0.022851,0.022851,0.018706,0.016456,NaN,0.0,0.953190,0.780483
2,CellLENS_Lite,level_3,0.666797,0.775340,0.703598,0.486879,0.461948,0.597235,0.592547,0.974513,...,0.020044,0.016026,0.029031,0.029031,0.038581,0.027781,NaN,0.0,0.943752,0.729602
3,FuseSOM_15,level_1,0.806554,NaN,0.824054,0.602036,0.814648,0.712393,0.710701,0.995601,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.0,1.000000,0.803562
4,FuseSOM_15,level_2,0.769324,NaN,0.781365,0.600316,0.772793,0.685472,0.681155,0.936103,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.0,1.000000,0.778476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,tribus,level_2,0.689865,NaN,0.683836,0.531833,0.266300,0.559993,0.550454,0.796052,...,0.059058,0.061256,0.034477,0.034477,0.051491,0.041068,NaN,0.0,0.800445,0.657232
92,tribus,level_3,0.611721,0.664520,0.576541,0.385603,0.296823,0.463314,0.455693,0.865220,...,0.049136,0.041565,0.031878,0.031878,0.138165,0.056139,NaN,0.0,0.798031,0.620362
93,xgboost_default_StratifiedGroupKFold,level_1,0.947002,NaN,0.946750,0.907800,0.912472,0.915787,0.914977,0.991828,...,0.011789,0.012027,0.003910,0.003910,0.000678,0.000676,0.0,0.0,0.979361,0.939827
94,xgboost_default_StratifiedGroupKFold,level_2,0.938819,NaN,0.938269,0.905050,0.914320,0.912231,0.911607,0.995858,...,0.010951,0.010503,0.003531,0.003531,0.000628,0.000626,0.0,0.0,0.979888,0.938033


In [8]:
# Save the final results to a CSV file with ';' as the separator
final_results.to_csv(os.path.join(base_path, "final_results.csv"), index=False, sep=';')

In [101]:
# Metric categories and their sub-metrics with weights
metrics_wt = {
    "Classification Performance": {
        "macro_f1_mean": 0.1,
        "mcc_mean": 0.10,
        "kappa_mean": 0.1,
        "f1_weighted_mean": 0.06,
        "accuracy_mean": 0.04
    },
    "Cell Type Composition": {
        "r2_mean": 0.06,
        "pearson_corr_mean": 0.06,
        "ari_mean": 0.09,
        "nmi_mean": 0.09
    },
    "Stability": {
        "stability": 0.2
    },
    "Scalability": {
        "runtime": 0.10
    }
}

# calculate the weighted score for each method, only if the metric is present in final_results
def calculate_weighted_score(row, metrics_wt):
    score = 0
    for category, metrics in metrics_wt.items():
        for metric, weight in metrics.items():
            if metric in row:
                score += row[metric] * weight
    return score

# Apply the weighted score calculation
final_results['weighted_score'] = final_results.apply(lambda row: calculate_weighted_score(row, metrics_wt), axis=1)

In [9]:
final_results

,method,level,f1_weighted_mean,hierarchial_f1_mean,accuracy_mean,macro_f1_mean,mcc_mean,kappa_mean,r2_mean,pearson_corr_mean,...,kappa_std,r2_std,pearson_corr_std,ari_std,nmi_std,jsd_std,jsd_scaled_std,kl_divergence_std,kl_scaled_std,stability
0,celllens_LITE,level_1,0.788653,NaN,0.803389,0.469044,0.684407,0.679668,0.963801,0.981643,...,0.027585,0.029180,0.014906,0.031867,0.029341,0.020956,0.020956,0.020155,0.017381,0.840763
1,celllens_LITE,level_2,0.755216,NaN,0.777380,0.479881,0.672814,0.668232,0.973147,0.986401,...,0.024798,0.027795,0.014147,0.022455,0.023145,0.023286,0.023286,0.030208,0.024901,0.810616
2,celllens_LITE,level_3,0.659848,0.766450,0.698724,0.446223,0.591576,0.586953,0.971893,0.985812,...,0.028483,0.017932,0.009128,0.022721,0.023710,0.029634,0.029634,0.047499,0.033332,0.766545
3,celllens_full,level_1,0.766359,NaN,0.775488,0.439271,0.650596,0.641313,0.907938,0.952580,...,0.019974,0.048689,0.025720,0.017163,0.011848,0.020896,0.020896,0.033841,0.027113,0.881787
4,celllens_full,level_2,0.709971,NaN,0.733022,0.427790,0.612692,0.604567,0.923177,0.960582,...,0.020131,0.045940,0.023949,0.026425,0.032952,0.036462,0.036462,0.055323,0.042674,0.861932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,tribus,level_2,0.703929,NaN,0.695485,0.539610,0.584846,0.572785,0.739327,0.857539,...,0.035974,0.117236,0.070298,0.040377,0.025030,0.022567,0.022567,0.042518,0.031922,0.777785
83,tribus,level_3,0.633979,0.678457,0.597261,0.429557,0.496122,0.488219,0.870576,0.932585,...,0.034828,0.060071,0.032814,0.029950,0.017437,0.027970,0.027970,0.138567,0.057350,0.760949
84,xgboost_default_StratifiedGroupKFold,level_1,0.947002,NaN,0.946750,0.907800,0.915787,0.914977,0.991828,0.995905,...,0.006884,0.002765,0.001389,0.011789,0.012027,0.003910,0.003910,0.000678,0.000676,0.958722
85,xgboost_default_StratifiedGroupKFold,level_2,0.938819,NaN,0.938269,0.905050,0.912231,0.911607,0.995858,0.997927,...,0.006161,0.001860,0.000932,0.010951,0.010503,0.003531,0.003531,0.000628,0.000626,0.959776


## Get limited cell type scores

In [14]:
# define the limited cell types for evaluation
limited_cell_types = [
    'B_cell', 'CD4+_T_cell', 'CD8+_T_cell', 'Neutrophil', 'Dendritic_cell', 'Treg'
]


In [15]:
def calculate_metrics_for_limited(methods, levels, base_path):
    all_results = []

    for method in tqdm(methods, desc="Methods"):
        try:
            for level in levels:
                print(f"Processing method: {method}, level: {level}")
                path = os.path.join(base_path, method, "level3")
                
                if os.path.exists(path):
                    files = [f for f in os.listdir(path) if f.startswith("predictions") and f.endswith(".csv")]
                else:
                    print(f"Path {path} does not exist. Skipping...")
                    files = []

                for file in files:
                    file_path = os.path.join(path, file)
                    if not os.path.exists(file_path):
                        print(f"File {file_path} does not exist. Skipping...")
                        continue
                    df = pd.read_csv(file_path)

                    df = df[df['true_phenotype'].isin(limited_cell_types)]

                    fold_name = os.path.splitext(file)[0]
                    
                    if level == 'level_3':
                        true_label = df['true_phenotype']
                        predictions = df["predicted_phenotype"]
                    else:
                        df[f'mapped_{level}_cell_type'] = df['predicted_phenotype'].apply(lambda x: map_predicted_to_levels(x)[f'{level}_cell_type'])
                        mask_na = df[f'mapped_{level}_cell_type'].isna()
                        df.loc[mask_na, f'mapped_{level}_cell_type'] = df.loc[mask_na, 'predicted_phenotype']

                        if f'{level}_cell_type' not in df.columns:
                            true_label = df['true_phenotype'].apply(lambda x: map_predicted_to_levels(x)[f'{level}_cell_type'])
                        else:
                            true_label = df[f'{level}_cell_type']
                        
                        predictions = df[f'mapped_{level}_cell_type'] 

                    f1 = f1_score(true_label, predictions, average='weighted')
                    accuracy = (true_label == predictions).mean()
                    macro_f1 = f1_score(true_label, predictions, average='macro')
                    mcc = matthews_corrcoef(true_label, predictions)
                    kappa = cohen_kappa_score(true_label, predictions)

                    if level == 'level_3':
                        ancestor_map = parse_tree_file("cell_type_hierarchy.txt")
                        hierarchical_f1 = hierarchical_f1_score(true_label, predictions, ancestor_map)
                    else:
                        hierarchical_f1 = None
                    
                    cell_type_distribution = calculate_cell_type_distribution(df, predictions, true_label)
                    r2, pearson_corr = calculate_r2_and_pearson(cell_type_distribution)
                    ari = adjusted_rand_score(true_label, predictions)
                    nmi = normalized_mutual_info_score(true_label, predictions)
                    kl_divergence = sum(cell_type_distribution['predicted_percentage'] / 100 * np.log2((cell_type_distribution['predicted_percentage'] + 1e-9) / (cell_type_distribution['true_percentage'] + 1e-9)))
                    scaled_kl_mean = 1 / (1 + kl_divergence)
                    jensen = jensenshannon(cell_type_distribution['predicted_percentage'] / 100, cell_type_distribution['true_percentage'] / 100, base=2)
                    jensen_scaled = 1 - jensen
                    g_mean = gmean_score(true_label, predictions)
                    
                    times_path1 = os.path.join(path, "fold_times.txt")
                    times_path2 = os.path.join(base_path,method, "fold_times.txt")
                    if os.path.exists(times_path1):
                        average_train_time, average_inference_time = calculate_time(times_path1)
                    elif os.path.exists(times_path2):
                        average_train_time, average_inference_time = calculate_time(times_path2)
                    else:
                        average_train_time = None
                        average_inference_time = None

                    all_results.append({
                        'method': method,
                        'level': level,
                        'fold': fold_name,
                        'f1_weighted': f1,
                        'hierarchical_f1': hierarchical_f1,
                        'accuracy': accuracy,
                        'macro_f1': macro_f1,
                        'g_mean': g_mean,
                        'mcc': mcc,
                        'kappa': kappa,
                        'r2': r2,
                        'pearson_corr': pearson_corr,
                        'ari': ari,
                        'nmi': nmi,
                        'jsd': jensen,
                        'jsd_scaled': jensen_scaled,
                        'kl_divergence': kl_divergence,
                        'kl_scaled': scaled_kl_mean,
                        'train_time': average_train_time,
                        'inference_time':average_inference_time
                    })
            print(f"Finished processing method: {method}")
        except Exception as e:
            print(f"Error encountered for method {method}: {e}. Skipping this method.")
            continue

    return pd.DataFrame(all_results)


In [16]:
# minimal cell types results
base_path = "../results/cHL_2_MIBI"
levels = ['level_1', 'level_2', 'level_3']

#make a list of folders in the base_path
methods = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
# Calculate metrics for all methods and levels
results = calculate_metrics_for_limited(methods, levels, base_path)
# Get the average of the metrics for each method and level, excluding the 'fold' column
average_results = results.drop(columns=["fold"]).groupby(['method', 'level'], as_index=False).mean()
# Get the standard deviation of the metrics for each method and level
std_results = results.drop(columns=["fold"]).groupby(['method', 'level']).std().reset_index()
# Merge the average and standard deviation results
final_results = pd.merge(average_results, std_results, on=['method', 'level'], suffixes=('_mean', '_std'))

# # Rename the columns for clarity
# final_results.columns = ['method', 'level', 'f1_weighted_mean', 'hierarchial_f1_mean', 'accuracy_mean', 'macro_f1_mean', 
#                          'mcc_mean', 'kappa_mean', 'r2_mean', 'pearson_corr_mean', 'ari_mean', 'nmi_mean',
#                          'jsd_mean', 'jsd_scaled_mean', 'kl_divergence_mean', 'kl_scaled_mean',
#                           'f1_weighted_std', 'hierarchial_f1_std','accuracy_std', 'macro_f1_std', 
#                           'mcc_std', 'kappa_std', 'r2_std', 'pearson_corr_std', 'ari_std', 'nmi_std',
#                           'jsd_std', 'jsd_scaled_std', 'kl_divergence_std', 'kl_scaled_std']

# calculate a stability metric for the methods where s = (1 - std/stability_thresh)
stability_thresh = 0.2
final_results['stability'] = 1 - (final_results['f1_weighted_std'] / stability_thresh)
# set to 0 if negative
final_results.loc[final_results['stability'] < 0, 'stability'] = 0

# Save the final results to a CSV file with ';' as the separator
final_results.to_csv(os.path.join(base_path, "minimal_results.csv"), index=False, sep=';')

Methods:   0%|          | 0/35 [00:00<?, ?it/s]

Processing method: phenograph_80, level: level_1
Processing method: phenograph_80, level: level_2
Processing method: phenograph_80, level: level_3


Methods:   3%|▎         | 1/35 [00:54<30:54, 54.55s/it]

Finished processing method: phenograph_80
Processing method: maps, level: level_1
Processing method: maps, level: level_2
Processing method: maps, level: level_3


Methods:   6%|▌         | 2/35 [01:31<24:18, 44.21s/it]

...

Finished processing method: nimbus_flowsom
Processing method: ribca, level: level_1
Processing method: ribca, level: level_2
Processing method: ribca, level: level_3


Methods:  97%|█████████▋| 34/35 [26:42<00:53, 53.03s/it]

Finished processing method: ribca
Processing method: CellLENS_Lite, level: level_1
Processing method: CellLENS_Lite, level: level_2
Processing method: CellLENS_Lite, level: level_3


Methods: 100%|██████████| 35/35 [27:36<00:00, 47.34s/it]

Finished processing method: CellLENS_Lite


## Get the hierarchy mappings

In [ ]:
# get the hierarchy mappings from the qt files
qt_path = "../data/all_qt"
#read all csv files in the qt_path
qt_files = [f for f in os.listdir(qt_path) if f.endswith('.csv')]
hierarchy_mappings = {}

for qt_file in qt_files:
    qt_df = pd.read_csv(os.path.join(qt_path, qt_file))
    # Create a hierarchical mapping from cell_type to level_2 and level_1 for this file
    mapping = qt_df.drop_duplicates(subset=['cell_type', 'level_2_cell_type', 'level_1_cell_type'])[
        ['cell_type', 'level_2_cell_type', 'level_1_cell_type']
    ].set_index('cell_type').to_dict(orient='index')
    # Update the global mapping without including the qt file name
    hierarchy_mappings.update(mapping)

# Remove duplicate cell types in the hierarchy_mappings (if any)
unique_cell_types = set()
for cell_type in list(hierarchy_mappings.keys()):
    if cell_type not in unique_cell_types:
        unique_cell_types.add(cell_type)
    else:
        del hierarchy_mappings[cell_type]

#save the hierarchy mappings to a file
with open('hierarchy_mappings.pkl', 'wb') as f:
    import pickle
    pickle.dump(hierarchy_mappings, f)

#read the hierarchy mappings from the file
with open('hierarchy_mappings.pkl', 'rb') as f:
    import pickle
    hierarchy_mappings = pickle.load(f)

        

{'Macrophage': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_cell_type': 'Immune'},
 'Plasma_cell': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'Neutrophil': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_cell_type': 'Immune'},
 'Adipocyte': {'level_2_cell_type': 'Adipocyte',
  'level_1_cell_type': 'Stromal'},
 'CD8+_T_cell': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'Endothelial': {'level_2_cell_type': 'Vascular',
  'level_1_cell_type': 'Stromal'},
 'CD4+_T_cell': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'undefined': {'level_2_cell_type': 'undefined',
  'level_1_cell_type': 'undefined'},
 'Osteoblast': {'level_2_cell_type': 'Bone', 'level_1_cell_type': 'Stromal'},
 'HSCs': {'level_2_cell_type': 'HSCs', 'level_1_cell_type': 'HSCs'},
 'Osteocyte': {'level_2_cell_type': 'Bone', 'level_1_cell_type': 'Stromal'},
 'Dendritic_cell': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_

In [ ]:
# Create a hierarchical mapping from true_phenotype to level_2 and level_1
hierarchy_mapping = pred.drop_duplicates(subset=['true_phenotype', 'level_2_cell_type', 'level_1_cell_type'])[
    ['true_phenotype', 'level_2_cell_type', 'level_1_cell_type']
].set_index('true_phenotype').to_dict(orient='index')

# Example usage: get level_2 and level_1 for a predicted phenotype
def map_predicted_to_levels(predicted_phenotype):
    return hierarchy_mappings.get(predicted_phenotype, {'level_2_cell_type': None, 'level_1_cell_type': None})

# Apply the mapping to the predicted phenotypes
pred['mapped_level_2_cell_type'] = pred['predicted_phenotype'].apply(lambda x: map_predicted_to_levels(x)['level_2_cell_type'])
pred['mapped_level_1_cell_type'] = pred['predicted_phenotype'].apply(lambda x: map_predicted_to_levels(x)['level_1_cell_type'])

pred

## visualization

In [8]:
hierarchy_mapping

{'CD4+_T_cell': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'unedfined': {'level_2_cell_type': 'unedfined',
  'level_1_cell_type': 'unedfined'},
 'Dendritic_cell': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_cell_type': 'Immune'},
 'CD8+_T_cell': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'M2_Macrophage': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_cell_type': 'Immune'},
 'B_cell': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'Treg': {'level_2_cell_type': 'Lymphoid_immune',
  'level_1_cell_type': 'Immune'},
 'Endothelial': {'level_2_cell_type': 'Vascular',
  'level_1_cell_type': 'Stromal'},
 'M1_Macrophage': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_cell_type': 'Immune'},
 'Cancer': {'level_2_cell_type': 'Cancer', 'level_1_cell_type': 'Cancer'},
 'Neutrophil': {'level_2_cell_type': 'Myeloid_immune',
  'level_1_cell_type': 'Immune'},
 'NK_cell': {'level_2_cell_type': 'L

In [40]:
from collections import defaultdict

# Group cell types by their level_1 and level_2 phenotypes

grouped_hierarchy = defaultdict(lambda: defaultdict(list))

for cell_type, levels in hierarchy_mappings.items():
    lvl1 = levels['level_1_cell_type']
    lvl2 = levels['level_2_cell_type']
    grouped_hierarchy[lvl1][lvl2].append(cell_type)

# Print the grouped hierarchy in a readable format
for lvl1, lvl2_dict in grouped_hierarchy.items():
    print(lvl1)
    for lvl2, cell_types in lvl2_dict.items():
        print(f"\t{lvl2}")
        for ct in cell_types:
            print(f"\t\t{ct}")

Immune
	Myeloid_immune
		Macrophage
		Neutrophil
		Dendritic_cell
		Monocyte
		M1_Macrophage
		Mast_cell
		M2_Macrophage
		Mature Myeloid
		Non-classical_Monocyte
		Plasmacytoid_dendritic_cell
		Myeloid
		M2a_Macrophage
		M1a_Macrophage
		M1b_Macrophage
		M2c_Macrophage
		M2b_Macrophage
		Placental_Mac
	Lymphoid_immune
		Plasma_cell
		CD8+_T_cell
		CD4+_T_cell
		B_cell
		NK_cell
		Treg
		Cytotoxic_CD8+_T_cell
		B_cells
		Immature_B_cell
		CD4_Treg
		CD8_Treg
		ki67_CD4
		ki67_CD8
		CD8_CXCL13
		CD4_CXCL13
		IDO_CD4
		IDO_CD8
		T_cell
		Cytotoxic_CD4+_T_cell
		BnT
		NK1
		NK_T_cell
		NK2
		NK3
		NK4
Stromal
	Adipocyte
		Adipocyte
	Vascular
		Endothelial
		Lymphatic
		SEC
		AEC
		VSMC
		Blood
		HEV
	Bone
		Osteoblast
		Osteocyte
		Osteoclast
	Vimentin high
		Vimentin high
	Fibroblast
		SMA+_Fibroblast
		CD10_CAF
		FN_Cdh11_mCAF
		Fibroblast
		vCAF
		CXCL13_CAF
		CD73_CAF
		CCL21_CAF
		IDO_CAF
		CD34_CAF
		CA9_CD10_CAF
	Stroma
		Stroma
	Myofibroblasts
		Myofibroblasts
	muscle
		muscle
	Gl